In [111]:
import os
import pandas as pd

# Set the base directory
base_dir = './Data/'

Accidents = []

# Loop over the directories
for year in ['2019', '2020', '2021']:
    # Set the current directory
    current_dir = base_dir + year + '/'


    # Loop over the files in the current directory
    for file in os.listdir(current_dir):
        dfusagers = pd.read_csv(current_dir + "usagers-" + year + ".csv", sep=";")
        dfcaracteristiques = pd.read_csv(current_dir + "caracteristiques-" + year + ".csv", sep=";")
        dflieux = pd.read_csv(current_dir + "lieux-" + year + ".csv", sep=";")
        dfvehicles = pd.read_csv(current_dir + "vehicules-" + year + ".csv", sep=";")
       


        #join data frames for a given year
        result = pd.merge(dfusagers, dfcaracteristiques, on='Num_Acc')
        result = pd.merge(result, dflieux, on='Num_Acc')
        result = pd.merge(result, dfvehicles,  on=['Num_Acc','id_vehicule'])
     
    
        #Append data frames in a list 
        Accidents.append(result)
        
         
        
        
        
#Final data frame of accidents 2019-2021
dfAccidents = Accidents[0]

for i in range(11):
    dfAccidents = pd.concat([dfAccidents, Accidents[i+1]])
    

In [128]:
#Evolution annuelle du nombre d'accidents par région et departément
columns_to_display = ['Num_Acc', 'an','dep','com','agg'] 

print(dfAccidents[columns_to_display].head())

        Num_Acc    an dep    com  agg
0  201900000001  2019  93  93053    1
1  201900000001  2019  93  93053    1
2  201900000001  2019  93  93053    1
3  201900000002  2019  93  93066    1
4  201900000003  2019  92  92036    1


In [129]:
result = dfAccidents.groupby(['an','agg'])['Num_Acc'].size()
print(result)

an    agg
2019  1      200744
      2      331164
2020  1      157708
      2      263472
2021  1      193616
      2      322996
Name: Num_Acc, dtype: int64
